# Reranking Opensearch Results with Cohere
This is the example companion notebook to the blog post Reranking Opensearch Results with Cohere. Here we provide all of the code and test data to try out Cohere's reranking api integrated in Opensearch.

The only thing you need to run the code is a Cohere api key, which you will be prompted for when running the notebook.

## Initial Setup

In [ ]:
# install dependencies
# wget will download opensearch, opensearch-py is the python client for opensearch, and beir will be where we retrieve our test data
!pip install wget opensearch-py beir

In [ ]:
# download opensearch 2.12 and adjust some settings

!wget https://artifacts.opensearch.org/releases/bundle/opensearch/2.12.0/opensearch-2.12.0-linux-x64.tar.gz
!tar -xvf opensearch-2.12.0-linux-x64.tar.gz

!sudo swapoff -a

with open('/etc/sysctl.conf', 'a') as writefile:
    writefile.write("vm.max_map_count=262144\n")
    writefile.write("plugins.security.disabled: true\n")

with open('/etc/sysctl.conf', 'r') as readfile:
    print(readfile.read())

with open('./opensearch-2.12.0/config/opensearch.yml', 'a') as writefile:
    writefile.write("plugins.security.disabled: true\n")

with open('./opensearch-2.12.0/config/opensearch.yml', 'r') as readfile:
    print(readfile.read())


In [ ]:
# setup local perms to run opensearch

!sudo chown -R daemon:daemon opensearch-2.12.0/

In [ ]:
# start opensearch

%%bash --bg
sudo -H -u daemon opensearch-2.12.0/bin/opensearch

In [ ]:
# give opensearch time to start

!sleep 30

In [ ]:
# check opensearch connection

!curl -X GET http://localhost:9200

In [ ]:
from opensearchpy import OpenSearch

# create opensearch client

host = 'localhost'
port = 9200

# Create the client with ssl and auth disabled, NOT to be used for production!
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    timeout=500
)

print(client.info())

In [ ]:
from getpass import getpass

# getpass allows you to input your api key without printing it for the world to see.
cohere_api_key = getpass("paste your cohere api key here")

## Register a Cohere Rerank Model

### Create an ML Connector

In [ ]:
import requests

url = "http://localhost:9200/_plugins/_ml/connectors/_create"
headers = {
    'Content-Type': 'application/json'
}
data = {
    "name": "cohere-rerank",
    "description": "The connector to Cohere reanker model",
    "version": "1",
    "protocol": "http",
    "credential": {
        "cohere_key": cohere_api_key
    },
    "parameters": {
        "model": "rerank-english-v2.0"
    },
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": "https://api.cohere.ai/v1/rerank",
            "headers": {
                "Authorization": "Bearer ${credential.cohere_key}"
            },
            "request_body": "{ \"documents\": ${parameters.documents}, \"query\": \"${parameters.query}\", \"model\": \"${parameters.model}\", \"top_n\": ${parameters.top_n} }",
            "pre_process_function": "connector.pre_process.cohere.rerank",
            "post_process_function": "connector.post_process.cohere.rerank"
        }
    ]
}

response = requests.post(url, headers=headers, json=data)
print(response)
print(response.json())
connector_id = response.json()['connector_id']

### Register and Deploy the Model

In [ ]:
url = "http://localhost:9200/_plugins/_ml/models/_register?deploy=true"
headers = {
    'Content-Type': 'application/json'
}
data = {
    "name": "cohere rerank model",
    "function_name": "remote",
    "description": "test rerank model",
    "connector_id": connector_id
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)
print(response.json())
task_id = response.json()['task_id']
model_id = response.json()['model_id']

### Test the Model

In [ ]:
import json

url = "http://localhost:9200/_plugins/_ml/models/hB9kkZIBrwGp_yq1pS5O/_predict"
headers = {
    'Content-Type': 'application/json'
}
data = {
  "parameters": {
    "query": "Who is the main character of Star Wars?",
    "documents": [
      "Jar-Jar Binks is a comical, possibly secret sith character in Star Wars.",
      "Darth Vader, aka Anakin Skywalker is the main antagonist of the original Star Wars trilogy.",
      "Luke Skywalker is the main protagonist of the original Star Wars trilogy.",
      "Emperor Palpatine is arguably the main antogonist as he is the main sith lord."
    ],
    "top_n": 4
  }
}

response = requests.post(url, headers=headers, json=data)
print(json.dumps(response.json()["inference_results"], indent=4))

## Configure a Reranking (Search) Pipeline

In [ ]:
url = "http://localhost:9200/_search/pipeline/rerank_pipeline_cohere"
headers = {
    'Content-Type': 'application/json'
}
data = {
    "description": "Pipeline for reranking with Cohere Rerank model",
    "response_processors": [
        {
            "rerank": {
                "ml_opensearch": {
                    "model_id": model_id
                },
                "context": {
                    "document_fields": ["title", "txt"],
                }
            }
        }
    ]
}

response = requests.put(url, headers=headers, json=data)

print(response.status_code)
print(response.json())

## Download and Index Test Data
We're using the scifact dataset from the BEIR project.

In [ ]:
import tqdm

from typing import List, Dict
from opensearchpy.helpers import bulk
from beir import util
from beir.datasets.data_loader import GenericDataLoader


dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = "datasets"
data_path = util.download_and_unzip(url, out_dir)
corpus, _, _ = GenericDataLoader(data_path).load(split="test")

hostname = "localhost:9200"
index_name = "scifact"

# define a function that will index the docs from the dataset corpus
def index_corpus(
    corpus: Dict[str, Dict[str, str]],
    index_name: str,
    es_client: OpenSearch,
):
    """
    Pushing documents over to our index

    Args:
        `corpus`: The corpus of the dataset we have selected. It's a Huggingface dataset with the three fields (`_id`, `title`, `text`)
        `index_name`: The name of the Elasticsearch index
        `es_client`: An instance of a Python Elasticsearch client
    Returns:
        None
    """

    def get_iterable():
        for _id, value in corpus.items():
          doc = {
              "_id": str(_id),
              "_op_type": "index",
              "refresh": "wait_for",
              "title": value["title"],
              "txt": value["text"],
          }
          yield doc

    # and bulk index them
    bulk(client=es_client, index=index_name, actions=get_iterable(), max_retries=3)

    # making sure that the index has been refreshed
    es_client.indices.refresh(index=index_name)

# index the docs
index_corpus(corpus, index_name, client)

In [ ]:
# define a function to use in printing search results
def format_response(response):
  hits = []

  for hit in response["hits"]["hits"]:
      response_object = {
          "id": hit['_id'],
          "score": hit['_score'],
          "title": hit['_source']['title'],
          "text": format(hit['_source']['txt'])
      }
      hits.append(response_object)

  print(json.dumps(hits, indent=2))

## Query Opensearch

### Establish Baseline BM25
First we'll create a basic multi_match query that will score the documents with BM25.

In [ ]:
query_text = 'A total of 1,000 people in the UK are asymptomatic carriers of vCJD infection.'

In [ ]:
res = client.search(
    index="scifact",
    body={
    "query": {
        "multi_match": {
            "query": query_text,
            "type": "best_fields",
            "fields": [
                "title",
                "txt"
            ],
            "tie_breaker": 0.5
        }
    },
    "size": 10
}
)

format_response(res)

### Use Rerank Search Pipeline
Now we'll create a query using our rerank search pipeline.

In [ ]:
res = client.search(
    index="scifact",
    search_pipeline="rerank_pipeline_cohere",
    body={
    "query": {
        "multi_match": {
            "query": query_text,
            "type": "best_fields",
            "fields": [
                "title",
                "txt"
            ],
            "tie_breaker": 0.5
        }
    },
    "size": 10,
    "ext": {
        "rerank": {
            "query_context": {
                "query_text": query_text
            }
        }
    }
}
)

format_response(res)

### Comparing Results
Our very scientific example query comes directly from the scifi dataset: “A total of 1,000 people in the UK are asymptomatic carriers of vCJD infection.” The expected result is the document with id “13734012” and title “Prevalent abnormal prion protein in human appendixes after bovine spongiform encephalopathy epizootic: large scale survey”.

We can see this document was ranked second in our normal search and first in our reranked results. While this is a nice anecdotal demonstration of the improved relevance, we calculated several relevance metrics for all of the queries in the scifi dataset, which demonstrates a consistent improvement in all scores as seen below.


## Performance Metrics
__Relevance (Normalized Discounted Cumulative Gain)__

|          | SBERT   | TAS-B   | BM25 + CE | BM25    | BM25 + Cohere |
|----------|---------|---------|-----------|---------|---------------|
| NDCG@1   | 0.42333 | 0.44667 |    0.5733 | 0.57667 |       0.62667 |
| NDCG@3   | 0.48416 | 0.50432 |    0.6314 | 0.63658 |       0.69593 |
| NDCG@5   | 0.48416 | 0.52853 |     0.652 | 0.66524 |        0.7141 |
| NDCG@10  | 0.53789 | 0.55485 |     0.672 | 0.69064 |       0.73495 |
| NDCG@100 | 0.57592 | 0.58717 |     0.678 | 0.71337 |       0.75241 |

- SBERT refers to an exact k-nn match using the sbert msmarco-distilbert-base-v3 model.
- TAS-B is exact match with sbert msmarco-distilbert-base-tas-b model.
- BM25 + CE is the base bm25 results reranked with the ms-marco-electra-base SBERT cross-encoder.
- BM25 is the base performance of a multi_match query in Opensearch.
- Finally, BM25 + Cohere is the performance of Opensearch when using the Cohere rerank pipeline.

__Latency__

| Latency | BM25    | BM25 + Cohere | BM25 + CE CPU |
|---------|---------|---------------|---------------|
| Average | 14.26ms | 214.03ms      | 8745.06ms     |
| P50     | 13.06ms | 150.35ms      | 8527.61ms     |
| P90     | 19.73ms | 406.88ms      | 12713.91ms    |
| P99     | 31.51ms | 870.71ms      | 15520.40ms    |

As you can see, relevance (as measured by nDCG is improved) at the cost of some latency (which makes sense because we’re reaching out across the internet to the cohere api for each request.) However, the Cohere reranking pipeline provides an advantage over self-hosting a cross-encoder pipeline, where performance degrades significantly without proper processing power. In our tests, we ran the sbert cross-encoder on google collab’s CPU and you can see the latency reached unacceptable levels. To improve the performance, the production environment would likely need many high performance CPUs or even GPUs would be preferred.